In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-124267
Azure region: southcentralus
Subscription id: 8bb47da5-84b5-43cf-bd4a-97928e5c9b08
Resource group: aml-quickstarts-124267


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [16]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling({
    "C": uniform(0.01, 2.0),
    "max_iter": choice(50,75,100,150,200)
})

# Specify a Policy
# policy = ### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.2, evaluation_interval = 1, delay_evaluation = 5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###
est = SKLearn(
    source_directory='.',
    compute_target = amlcompute_cluster_name,
    entry_script = 'train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    estimator = est,
    hyperparameter_sampling = ps,
    policy = policy,
    primary_metric_name = "Accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 20,
    max_concurrent_runs = 4
)

print('hyperdrive_config created')

hyperdrive_config created


In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyper_run = exp.submit(hyperdrive_config)
RunDetails(hyper_run).show()
# hyper_run.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [19]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyper_run.get_best_run_by_primary_metric()
parameters = best_run.get_details()['runDefinition']['arguments']
joblib.dump(parameters, filename='outputs/best-hyperdrive.joblib')

model = best_run.register_model(model_name='best-hyperdrive', model_path='.')

In [21]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

ds = TabularDatasetFactory.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [23]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
# x, y = clean_data(### YOUR DATA OBJECT HERE ###)

x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
train_data = x_train.join(y_train)

In [26]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5)

In [27]:
# Submit your automl run

### YOUR CODE HERE ###

exp2 = Experiment(ws, 'udacity-project-automl')
auto_run = exp2.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_34e0ade0-ec7d-4871-b926-7b08a8e9bf8d

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = auto_run.get_output()
joblib.dump(fitted_model, filename='outputs/best-automl.joblib')

model = auto_run.register_model(model_name='best-automl')

In [ ]:
# Delete Clusters

aml_compute.delete()
compute_target.delete()